---

Feature Engineering

---

_The assertions and methodologies outlined in this notebook are substantiated by referenced scientific studies detailed in the README file._

### Load libraries and Data

In [8]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import sys
sys.path.append("../src")
from extract_features import *
from visualize_audio import *
from data_manager import *
from preprocess_data import *
from load_config import *
from split_silence_transformer import SplitSilenceTransformer
from mfcc_transformer import MfccTransformer

In [2]:
constants = load_constants_from_yaml('../constants.yml')

SAMPLING_RATING = constants["SAMPLING_RATING"]
FRAME_LENGTH_ENERGY = constants["FRAME_LENGTH_ENERGY"]
THRESHOLD_PERCENTAGE = constants["THRESHOLD_PERCENTAGE"]
MIN_SILENCE_DURATION = constants["MIN_SILENCE_DURATION"]
HOP_LENGTH = constants["HOP_LENGTH"]
SEGMENT_DURATION = constants["SEGMENT_DURATION"]
SEGMENT_OVERLAP = constants["SEGMENT_OVERLAP"]
N_MFCC = constants["N_MFCC"]
CONSIDERED_ACCENTS = constants["CONSIDERED_ACCENTS"]

In [3]:
df = load_audio_files("../data/raw/recordings/", sr=SAMPLING_RATING)
df = filter_data_based_on_accents(df=df, considered_accents=CONSIDERED_ACCENTS)

Trim silence from audio

In [4]:
split_tranformer=SplitSilenceTransformer(
    variables=['audio', 'labels'],
    sampling_rating=SAMPLING_RATING,
    threshold_percentage=THRESHOLD_PERCENTAGE,
    min_silence_duration=MIN_SILENCE_DURATION,
    frame_length_energy=FRAME_LENGTH_ENERGY,
    hop_length=HOP_LENGTH
)
split_tranformer.fit(df)

SplitSilenceTransformer(frame_length_energy=2048, hop_length=512,
                        min_silence_duration=1, sampling_rating=22050,
                        threshold_percentage=0.01,
                        variables=['audio', 'labels'])

In [6]:
df=split_tranformer.transform(df)

In [9]:
df.shape

(242, 2)

One of the most commonly used spectral feature representations is the Mel-frequency cepstral coefficients (MFCC). MFCC features are generally employed in automatic speech recognition (ASR) and accent recognition systems and are known to perform best in shallow models. Spectrograms, on the other hand, are more effective in deep models and are sometimes utilized in accent recognition. We will extract MFCCs using the Librosa library.

Extract MFCC features from trimmed audio data (not on segmented audio data because the function itself will split the audio data into segments).

In [10]:
mfcc_transformer=MfccTransformer(
    variables=["audio", "labels"],
    sampling_rating=SAMPLING_RATING, 
    n_mfcc=N_MFCC,
    duration=SEGMENT_DURATION,
    overlap=SEGMENT_OVERLAP
)

TypeError: MfccTransformer.__init__() missing 1 required positional argument: 'audio'

In [11]:
'''audio_data, labels = df["audio"], df["labels"]
mfccs = []
for audio in audio_data_trimmed:
    mfcc = compute_mfcc(
        audio, SAMPLING_RATING, n_mfcc=N_MFCC, duration=SEGMENT_DURATION, overlap=SEGMENT_OVERLAP
    )
    mfccs.append(mfcc)'''

'audio_data, labels = df["audio"], df["labels"]\nmfccs = []\nfor audio in audio_data_trimmed:\n    mfcc = compute_mfcc(\n        audio, SAMPLING_RATING, n_mfcc=N_MFCC, duration=SEGMENT_DURATION, overlap=SEGMENT_OVERLAP\n    )\n    mfccs.append(mfcc)'

In [ ]:
print(f"len(mfccs): {len(mfccs)}")
print(f"num_segments for mfccs[0]: {mfccs[0].shape[1]}")
print(f"num_segments for mfccs[5]: {mfccs[5].shape[1]}")

len(mfccs): 385
num_segments for mfccs[0]: 2318
num_segments for mfccs[5]: 2321


In [ ]:
# we want to split the mfccs into segments with the corresponding labels

# Initialize lists to store segments and their corresponding labels
segmented_mfccs = []
segmented_labels = []

# Iterate over each audio
for i, mfcc in enumerate(mfccs):
    # Get the shape of MFCCs and label for the current audio
    mfcc_shape = mfcc.shape
    label = labels_trimmed[i]

    # Extract the number of segments and the number of frames per segment
    num_segments = mfcc.shape[1]  # Second dimension of the MFCC shape

    # Iterate over each segment in the current audio
    for j in range(num_segments):
        # Extract the MFCCs for the current segment
        mfcc_segment = mfccs[i][:, j]

        # Append the segment and its corresponding label to the lists
        segmented_mfccs.append(mfcc_segment)
        segmented_labels.append(label)

# Convert the lists to NumPy arrays
# segmented_mfccs = np.array(segmented_mfccs)
# segmented_labels = np.array(segmented_labels)

print(f"len(segmented_mfccs): {len(segmented_mfccs)}")
print(f"len(segmented_labels): {len(segmented_labels)}")
print(f"segmented_mfccs[0]: {len(segmented_mfccs[0])}")
print(f"segmented_labels[0]: {segmented_labels[0]}")

len(segmented_mfccs): 941351
len(segmented_labels): 941351
segmented_mfccs[0]: 13
segmented_labels[0]: english
